In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
from src.fetch import fetch_raw
from src.preprocess import preprocess
from src.utils import df_na_vals

In [9]:
raw = fetch_raw()
df = preprocess(raw)

In [10]:
cpu_name = df.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = df.cpu_name.str.replace('Core™', '')
df.cpu_name = cpu_name

df['cores'] = (df.cpu_cores / df.num_gpus).fillna(0).astype(int)
df['ram'] = (df.cpu_ram / df.num_gpus).round().astype(int)
df['dph'] = (df.dph_base / df.num_gpus).round().astype(int)
df['disk'] = (df.disk_space / df.num_gpus).astype(int)
df['rented_fraq'] = '[' + df.num_gpus_rented.astype(str) + "/" + df.num_gpus.astype(str) + ']'
# df['rented'] = df.num_gpus_rented > 0
df['rented'] = df.num_gpus_rented == df.num_gpus
# df['min_fraq'] = df.min_chunk/df.num_gpus

# df.dlperf = df.dlperf.round(1)
# df.dlperf_per_dphtotal = df.dlperf_per_dphtotal.round(1)
# df.disk_bw = df.disk_bw.round()
# df.duration = pd.to_timedelta(df.duration * 1e9).dt.days
# df.inet_down = df.inet_down.fillna(0).astype(int)
# df.inet_up = df.inet_up.fillna(0).astype(int)
# df.inet_up_cost = (df.inet_up_cost * 1000).fillna(0).astype(int)
# df.inet_down_cost = (df.inet_down_cost * 1000).fillna(0).astype(int)

# df['isp'] = df.location.apply(lambda x: x.pop('isp', None) if not pd.isna(x) else None)

# df.end_date = (pd.to_datetime(df.end_date * 1e9) - pd.Timestamp.now()).dt.days
df.end_date = pd.to_datetime(df.end_date * 1e9)
# df.start_date = pd.to_datetime(df.start_date * 1e9)
# df.set_index('machine_id', inplace=True)

In [11]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'country']

df.query("dph >= 0.5 and gpu_name == 'RTX 4090'")[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
1920,RTX 4090,[2/4],2500,0,16,32,345,1592,1,US,...,2,12.2,535.129.03,3,8,0,47,AMD EPYC 7551P 32C Processor,H11SSL,US
3618,RTX 4090,[2/4],2000,0,7,31,375,2939,1,PT,...,2,12.0,525.105.17,3,16,0,117,Core™ i9-10940X,Pro WS X299 SAGE II,PT
3450,RTX 4090,[6/7],2000,0,9,36,724,2475,1,PT,...,0,12.0,525.105.17,4,16,0,250,AMD Ryzen TR PRO 5975WX 32Cs,Pro WS WRX80E,PT
3470,RTX 4090,[7/7],2000,0,9,36,817,2636,1,PT,...,2,12.0,525.105.17,4,16,0,240,AMD Ryzen TR PRO 5975WX 32Cs,Pro WS WRX80E,PT
1200,RTX 4090,[2/12],1589,0,21,42,476,4324,1,NO,...,3,12.4,550.54.14,4,8,1,122,AMD EPYC 7702 64C Processor,ROME2D32GM,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497,RTX 4090,[0/1],219,0,28,126,740,1310,1,CA,...,5,12.0,525.147.05,3,16,0,115,Xeon® E5-2690 v4,X99,CA
2183,RTX 4090,[2/2],210,0,14,16,350,1858,1,UA,...,0,12.2,535.129.03,3,16,0,113,Xeon® E5-2680 v4,X99A GODLIKE GAMING CARBON,UA
506,RTX 4090,[2/2],200,0,14,63,380,2746,2,US,...,0,12.2,535.171.04,3,16,0,111,Xeon® E5-2690 v4,212B,US
1503,RTX 4090,[2/2],200,0,14,63,375,1947,2,US,...,0,12.2,535.171.04,3,16,0,111,Xeon® E5-2690 v4,212B,US


In [12]:
df.query("num_gpus == 4 and gpu_name == 'RTX 4090'")[cols]

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
3,RTX 4090,[0/4],1400,0,24,63,410,2624,1,CN,...,0,12.2,535.171.04,4,16,0,182,AMD EPYC 7642 48C Processor,H12SSL,CN
10,RTX 4090,[4/4],270,0,0,0,370,3135,1,DE,...,0,12.2,535.154.05,3,8,0,0,,X99,DE
22,RTX 4090,[3/4],400,0,8,31,717,1739,1,US,...,2,12.4,550.78,3,16,0,117,Core™ i9-7960X,WS X299 SAGE,US
39,RTX 4090,[0/4],344,0,12,63,830,5971,4,ZA,...,30,12.2,535.161.08,4,16,0,224,AMD EPYC 7402 24C Processor,H12SSL,ZA
40,RTX 4090,[4/4],274,950,0,0,160,3514,2,TW,...,2,12.2,535.171.04,4,8,0,0,,MZ32,TW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3796,RTX 4090,[0/4],500,200,16,32,350,893,1,NO,...,10,12.2,535.161.07,4,16,0,226,AMD Ryzen TR PRO 5975WX 32Cs,Pro WS WRX80E,NO
3821,RTX 4090,[4/4],440,400,12,32,945,950,4,CA,...,20,12.4,550.54.14,3,16,0,113,Xeon® E5-2695 v2,X9DRi,CA
3823,RTX 4090,[4/4],500,200,16,32,302,3378,1,NO,...,10,12.2,535.154.05,4,16,0,244,AMD Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,NO
3825,RTX 4090,[3/4],500,200,16,32,357,3351,1,NO,...,10,12.2,535.154.05,4,16,0,242,AMD Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,NO


In [14]:
df[cols].query('country == "RU" and gpu_name == "RTX 4090"')

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
373,RTX 4090,[2/2],300,0,28,32,355,2502,1,RU,...,2,12.0,525.116.04,3,16,0,113,Xeon® E5-2680 v4,X99,RU
1109,RTX 4090,[4/4],300,0,8,47,630,5317,1,RU,...,0,12.4,550.54.14,4,16,0,180,AMD EPYC 7302P 16C Processor,ROMED8,RU
1203,RTX 4090,[6/6],300,0,2,5,53,4274,6,RU,...,0,12.4,550.54.14,4,16,0,172,11th Gen Core™ i9-11900K,H510 Pro BTC,RU
1238,RTX 4090,[1/2],259,500,6,32,325,881,1,RU,...,1,12.4,550.54.14,4,8,0,128,AMD Ryzen 5 5600 6C Processor,X570 AORUS MASTER,RU
1601,RTX 4090,[2/2],400,0,32,63,705,1887,1,RU,...,0,12.2,535.161.07,4,16,0,239,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
1679,RTX 4090,[4/4],320,0,12,32,122,4481,1,RU,...,0,12.4,550.54.14,4,16,0,228,AMD EPYC 7352 24C Processor,ROMED8,RU
1711,RTX 4090,[1/1],300,0,12,16,40,260,1,RU,...,1,12.2,535.154.05,4,16,0,169,AMD Ryzen 5 3600 6C Processor,B550 Pro4,RU
1840,RTX 4090,[2/2],400,0,32,63,2655,5034,1,RU,...,0,12.2,535.154.05,4,16,0,240,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
2032,RTX 4090,[2/2],400,0,32,63,2410,4161,1,RU,...,0,12.2,535.154.05,4,8,0,124,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
2086,RTX 4090,[2/2],400,0,32,63,635,1951,1,RU,...,0,12.2,535.161.07,4,16,0,242,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
